# 08 — SQL Insights & Project Summary

This final notebook uses the cleaned SQL table (`clean_sales_final`) to
generate **business-style insights** and wrap up the project.

**Queries:**
- Top makes by total units sold
- Overall monthly sales trend
- Fastest-growing makes by month-on-month percentage growth

This mirrors the kind of analysis a data analyst would present to stakeholders
after building the forecasting pipeline.


In [ ]:
import sqlite3
import pandas as pd

DB_PATH = "data/auto_sales.db"
conn = sqlite3.connect(DB_PATH)

In [ ]:
q_top_makes = """
SELECT Make, SUM(Units_Sold) AS Total_Sales
FROM clean_sales_final
GROUP BY Make
ORDER BY Total_Sales DESC
LIMIT 10;
"""
top_makes = pd.read_sql(q_top_makes, conn)
print("Top makes by total sales:")
display(top_makes)

In [ ]:
q_trend = """
SELECT strftime('%Y-%m', Year_Month) AS YearMonth,
       SUM(Units_Sold) AS Total_Units
FROM clean_sales_final
GROUP BY YearMonth
ORDER BY YearMonth;
"""
trend = pd.read_sql(q_trend, conn)
print("Overall monthly sales trend (tail):")
display(trend.tail(12))

In [ ]:
q_growth = """
WITH Monthly AS (
  SELECT Make, strftime('%Y-%m', Year_Month) AS ym, SUM(Units_Sold) AS monthly_units
  FROM clean_sales_final
  GROUP BY Make, ym
),
Growth AS (
  SELECT
    Make,
    ym,
    monthly_units,
    LAG(monthly_units) OVER (PARTITION BY Make ORDER BY ym) AS prev_units
  FROM Monthly
)
SELECT Make, ym AS YearMonth, monthly_units, prev_units,
       ROUND(100.0 * (monthly_units - prev_units) / prev_units, 2) AS growth_pct
FROM Growth
WHERE prev_units IS NOT NULL
ORDER BY growth_pct DESC
LIMIT 10;
"""
growth = pd.read_sql(q_growth, conn)
print("Fastest-growing makes (MoM % growth):")
display(growth)

In [ ]:
conn.close()
print("Finished SQL insights. Full pipeline complete.")